In [2]:
import glob
import numpy as np
import astropy.units as u
from astropy.time import Time
from astropy.constants import c

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter

%matplotlib inline

# Fitting velocities

In [3]:
def arc_curvature_veff(params, ydata, weights, true_anomaly,
                  vearth_ra, vearth_dec, vfit=False, modelonly=False):
    """
    arc curvature model

        ydata: arc curvature (vfit=False), or 
               1 / sqrt(arc curvature) (when vfit=True)
    """

    # ensure dimensionality of arrays makes sense
    if hasattr(ydata,  "__len__"):
        ydata = ydata.squeeze()
        weights = weights.squeeze()
        true_anomaly = true_anomaly.squeeze()
        vearth_ra = vearth_ra.squeeze()
        vearth_dec = vearth_dec.squeeze()

    kmpkpc = 3.085677581e16

    # Other parameters in lower-case
    d = params['d']  # pulsar distance in kpc
    try:
        s = params['s']  # fractional screen distance
    except:
        d_s = params['d_s'] # screen distance in kpc
        s = 1 - d_s / d
    dkm = d * kmpkpc  # kms

    veff_ra, veff_dec, vp_ra, vp_dec = \
        effective_velocity_annual(params, true_anomaly,
                                  vearth_ra, vearth_dec)

    psi = params['psi'] * np.pi / 180  # anisotropy angle
    vism_psi = params['vism_psi']  # vism in direction of anisotropy
    #veff2 = (veff_ra*np.sin(psi) + veff_dec*np.cos(psi) - vism_psi)**2
    veff = (veff_ra*np.sin(psi) + veff_dec*np.cos(psi) - vism_psi)
    veff2 = veff**2 * np.sign(veff)

    # Calculate curvature model
    model = dkm * s * (1 - s)/veff2  # in 1/(km * Hz**2)

    # Convert to 1/(m * mHz**2) for beta in 1/m and fdop in mHz

    if weights is None:
        weights = np.ones(np.shape(ydata))

    kmHz_to_kmskpc = 175660968.387
    model = np.sign(veff) / np.sqrt(abs(model)) 
    model = np.abs(model) * kmHz_to_kmskpc
    #model = model * kmHz_to_kmskpc

    if modelonly:
        return model

    return ((ydata - model) * weights).astype('float')

In [4]:
def get_earth_velocity(mjds, raj, decj):
    """
    Calculates the component of Earth's velocity transverse to the line of
    sight, in RA and DEC
    """

    from astropy.time import Time
    from astropy.coordinates import get_body_barycentric_posvel, SkyCoord
    from astropy import units as u
    from astropy.constants import au

    coord = SkyCoord('{0} {1}'.format(raj, decj), unit=(u.hourangle, u.deg))
    rarad = coord.ra.value * np.pi/180
    decrad = coord.dec.value * np.pi/180

    vearth_ra = []
    vearth_dec = []
    for mjd in mjds:
        time = Time(mjd, format='mjd')
        pos_xyz, vel_xyz = get_body_barycentric_posvel('earth', time)

        vx = vel_xyz.x.value
        vy = vel_xyz.y.value
        vz = vel_xyz.z.value

        vearth_ra.append(- vx * np.sin(rarad) + vy * np.cos(rarad))
        vearth_dec.append(- vx * np.sin(decrad) * np.cos(rarad) -
                          vy * np.sin(decrad) * np.sin(rarad) +
                          vz * np.cos(decrad))

    # Convert from AU/d to km/s
    vearth_ra = vearth_ra * au/1e3/86400
    vearth_dec = vearth_dec * au/1e3/86400

    return vearth_ra.value.squeeze(), vearth_dec.value.squeeze()


def get_true_anomaly(mjds, pars):
    """
    Calculates true anomalies for an array of barycentric MJDs and a parameter
    dictionary
    """

    PB = pars['PB']
    T0 = pars['T0']
    ECC = pars['ECC']
    PBDOT = 0 if 'PBDOT' not in pars.keys() else pars['PBDOT']

    nb = 2*np.pi/PB

    # mean anomaly
    M = nb*((mjds - T0) - 0.5*(PBDOT/PB) * (mjds - T0)**2)
    M = M.squeeze()

    # eccentric anomaly
    if ECC < 1e-4:
        print('Assuming circular orbit for true anomaly calculation')
        E = M
    else:
        E = fsolve(lambda E: E - ECC*np.sin(E) - M, M)

    # true anomaly
    U = 2*np.arctan2(np.sqrt(1 + ECC) * np.sin(E/2),
                     np.sqrt(1 - ECC) * np.cos(E/2))  # true anomaly
    if hasattr(U,  "__len__"):
        U[np.argwhere(U < 0)] = U[np.argwhere(U < 0)] + 2*np.pi
        U = U.squeeze()
    elif U < 0:
        U += 2*np.pi

    return U


def effective_velocity_annual(params, true_anomaly, vearth_ra, vearth_dec,
                              mjd=None):
    """
    Effective velocity with annual and pulsar terms
        Note: Does NOT include IISM velocity, but returns veff in IISM frame
    """
    # Define some constants
    v_c = 299792.458  # km/s
    kmpkpc = 3.085677581e16
    secperyr = 86400*365.2425
    masrad = np.pi/(3600*180*1000)

    # tempo2 parameters from par file in capitals
    if 'PB' in params.keys():
        A1 = params['A1']  # projected semi-major axis in lt-s
        PB = params['PB']  # orbital period in days
        ECC = params['ECC']  # orbital eccentricity
        OM = params['OM'] * np.pi/180  # longitude of periastron rad
        if 'OMDOT' in params.keys():
            omega = OM + params['OMDOT']*np.pi/180*(mjd-params['T0'])/365.2425
        else:
            omega = OM
        # Note: fifth Keplerian param T0 used in true anomaly calculation
        if 'KIN' in params.keys():
            INC = params['KIN']*np.pi/180  # inclination
        elif 'COSI' in params.keys():
            INC = np.arccos(params['COSI'])
        elif 'SINI' in params.keys():
            INC = np.arcsin(params['SINI'])
        else:
            print('Warning: inclination parameter (KIN, COSI, or SINI) ' +
                  'not found')

        if 'sense' in params.keys():
            sense = params['sense']
            if sense < 0.5:  # KIN < 90
                if INC > np.pi/2:
                    INC = np.pi - INC
            if sense >= 0.5:  # KIN > 90
                if INC < np.pi/2:
                    INC = np.pi - INC

        KOM = params['KOM']*np.pi/180  # longitude ascending node

        # Calculate pulsar velocity aligned with the line of nodes (Vx) and
        #   perpendicular in the plane (Vy)
        vp_0 = (2 * np.pi * A1 * v_c) / (np.sin(INC) * PB * 86400 *
                                         np.sqrt(1 - ECC**2))
        vp_x = -vp_0 * (ECC * np.sin(omega) + np.sin(true_anomaly + omega))
        vp_y = vp_0 * np.cos(INC) * (ECC * np.cos(omega) + np.cos(true_anomaly
                                                                  + omega))
    else:
        vp_x = 0
        vp_y = 0
        KOM = 0

    if 'PMRA' in params.keys():
        PMRA = params['PMRA']  # proper motion in RA
        PMDEC = params['PMDEC']  # proper motion in DEC
    else:
        PMRA = 0
        PMDEC = 0

    # other parameters in lower-case
    s = params['s']  # fractional screen distance
    d = params['d']  # pulsar distance in kpc
    d = d * kmpkpc  # distance in km

    pmra_v = PMRA * masrad * d / secperyr
    pmdec_v = PMDEC * masrad * d / secperyr

    # Rotate pulsar velocity into RA/DEC
    vp_ra = np.sin(KOM) * vp_x + np.cos(KOM) * vp_y
    vp_dec = np.cos(KOM) * vp_x - np.sin(KOM) * vp_y

    # find total effective velocity in RA and DEC
    veff_ra = s * vearth_ra + (1 - s) * (vp_ra + pmra_v)
    veff_dec = s * vearth_dec + (1 - s) * (vp_dec + pmdec_v)

    return veff_ra, veff_dec, vp_ra, vp_dec



In [ ]:
from lmfit import Parameters, fit_report, minimize
from lmfit import Minimizer
import lmfit 
import corner

data = np.loadtxt('fitV.txt')

y = data[:,1]
yerr = data[:,2]
mjd = data[:,0]

isort = np.argsort(mjd)
mjd = mjd[isort]
y = y[isort]
yerr = yerr[isort]

vfit = 1

params = Parameters()

# Pulsar sky position
params.add('RAJ', value=1.835345, vary=False)
params.add('DECJ', value=1.122301, vary=False)

# Orbital Parameters
params.add('A1', value=32.34, vary=False)
params.add('PB', value=67.8, vary=False)
params.add('ECC', value=7.5e-5, vary=False)
params.add('OM', value=176.2, vary=False)
params.add('T0', value=54303.6340, vary=False)

params.add('PMRA', value=4.9238, vary=False)
params.add('PMDEC', value=-3.9146, vary=False)
params.add('d', value=1.15, vary=False)

# Unknown parameters of screen
params.add('s', value=0.6, vary=True, min=0.0, max=1.0)  
params.add('vism_psi', value=8.0, vary=True)
params.add('psi', value=60., vary=True, min=0, max=180)

params.add('KIN', value=71.7, vary=True, min=0, max=180)
params.add('KOM', value=89.0,
           vary=True, min=0, max=360)

print('Getting Earth velocity')
vearth_ra, vearth_dec = get_earth_velocity(mjd, params['RAJ'].value, params['DECJ'].value)

print('Getting true anomaly')
U = get_true_anomaly(mjd, params)

veff_ra, veff_dec, vp_ra, vp_dec = \
    effective_velocity_annual(params, U,
                              vearth_ra, vearth_dec)


func = Minimizer(arc_curvature_veff, params,
                 fcn_args=(y, 1/yerr, U,
                           vearth_ra, vearth_dec, 
                           vfit))

print('Doing mcmc posterior sample')    
mcmc_results = func.emcee(steps=5000, burn=1000, nwalkers=100)
mcmc_unchanged = mcmc_results

lmfit.report_fit(mcmc_results)

truths = []
for var in mcmc_results.var_names:
    truths.append(mcmc_results.params[var].value)
labels = mcmc_results.var_names

corner.corner(mcmc_results.flatchain,
              labels=labels,
              truths=truths)
lmfit.report_fit(mcmc_results)
#plt.savefig('{0}_mcmc2013_{1}.pdf'.format(psrname, alabel))

"""
Now plot the fit
"""
curvature_model = arc_curvature_veff(mcmc_results.params, y, yerr, U, vearth_ra, vearth_dec,
                                vfit, modelonly=True)                              


plt.figure(figsize=(8,4))
plt.errorbar(mjd, y, yerr, fmt='.')
plt.plot(mjd, curvature_model)
plt.show()     

Getting Earth velocity
Getting true anomaly
Assuming circular orbit for true anomaly calculation
Doing mcmc posterior sample
